In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# Load the product data
product_data1 = pd.read_csv('product_data.csv')

product_data1.head()


,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [4]:
product_data=product_data1[:10000]
product_data.count()

index           10000
product         10000
category        10000
sub_category    10000
brand            9999
sale_price      10000
market_price    10000
type            10000
rating           6843
description      9958
dtype: int64

In [5]:
# Preprocess the data (e.g., lowercase, remove punctuation)
product_data['description'] = product_data['description'].str.lower()
product_data['description'] = product_data['description'].str.replace('[^\w\s]', '')

product_data['category'] = product_data['category'].str.lower()
product_data['category'] = product_data['category'].str.replace('[^\w\s]', '')

# Handle missing values (if any)
product_data['description'].fillna('', inplace=True)
product_data['category'].fillna('', inplace=True)

C:\Users\rkj80\AppData\Local\Temp\ipykernel_1252\1188690281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_data['description'] = product_data['description'].str.lower()
C:\Users\rkj80\AppData\Local\Temp\ipykernel_1252\1188690281.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  product_data['description'] = product_data['description'].str.replace('[^\w\s]', '')
C:\Users\rkj80\AppData\Local\Temp\ipykernel_1252\1188690281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [6]:
# Concatenate the 'description' and 'category' columns, Here we have used two column to make recommendation
product_data['content'] = product_data['description'] + ' ' + product_data['category']

# Convert the concatenated content into TF-IDF feature vectors
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(product_data['content'])


C:\Users\rkj80\AppData\Local\Temp\ipykernel_1252\4056277551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_data['content'] = product_data['description'] + ' ' + product_data['category']


In [7]:
# Compute the cosine similarity matrix
similarity_matrix = cosine_similarity(feature_matrix, feature_matrix)


In [11]:
def get_recommendations(product, top_n=5):
    # Find the index of the target product
    product_index = product_data[product_data['product'] == product].index[0]

    # Get the similarity scores for the target product
    similarity_scores = similarity_matrix[product_index]

    # Sort the similarity scores and get the indices of top-n similar products
    top_indices = similarity_scores.argsort()[:-top_n-1:-1]

    # Get the product IDs of the top-n similar products
    top_product_ids = product_data.iloc[top_indices]['product'].values

    return top_product_ids





In [12]:
# Example usage:
product = 'Water Bottle - Orange'
recommendations = get_recommendations(product)
print(recommendations)

['Water Bottle - Orange'
 'Premium Rectangular Plastic Container With Lid - Multicolour'
 'Premium Round Plastic Container With Lid - Yellow'
 'Plastic Container - Square, Pink'
 'Premium Square Plastic Container - Green']


In [14]:
#describtion
print(product_data[product_data['product']=='Water Bottle - Orange'][['description','category','content']],'\n\n')
print(product_data[product_data['product']=='Premium Rectangular Plastic Container With Lid - Multicolour'][['description','category']],'\n\n')
print(product_data[product_data['product']=='Premium Round Plastic Container With Lid - Yellow'][['description','category']],'\n\n')
print(product_data[product_data['product']=='Plastic Container - Square, Pink'][['description','category']],'\n\n')
print(product_data[product_data['product']=='Premium Square Plastic Container - Green'][['description','category']],'\n\n')

                                         description              category  \
1  each product is microwave safe without lid ref...  kitchen garden  pets   

                                             content  
1  each product is microwave safe without lid ref...   


                                            description              category
6163  these containers are microwave safe without li...  kitchen garden  pets 


                                            description              category
9546  these containers are microwave safe without li...  kitchen garden  pets 


                                            description              category
8588  each product is microwave safe but without a l...  kitchen garden  pets 


                                           description              category
125  these containers are microwave safe but withou...  kitchen garden  pets 




In [16]:
print(product_data[product_data['product']=='Water Bottle - Orange'][['description','category','category']],'\n\n')


                                         description              category  \
1  each product is microwave safe without lid ref...  kitchen garden  pets   

               category  
1  kitchen garden  pets   


